# Run C-star

In [5]:
!cat wio_toy_blueprint.yaml

name: wio_toy
description: wio toy domain for laptop
application: roms_marbl
state: draft
valid_start_date: 2012-01-01
valid_end_date: 2012-01-02

code:
  roms:
    location: https://github.com/CWorthy-ocean/ucla-roms.git
    branch: a338198af93a7b4cfa8f320a23f0f5623bc18152
  marbl:
    location: https://github.com/marbl-ecosys/MARBL.git
    branch: marbl0.45.0

  run_time:
    location: https://github.com/CWorthy-ocean/cstar_blueprint_roms_marbl_example.git
    branch: main
    filter:
      directory: wio-toy-domain/runtime_code
      files:
        - cson_roms-marbl_v0.1_wio-toy_10procs.in
        - marbl_in
        - marbl_tracer_output_list
        - marbl_diagnostic_output_list
  compile_time:
    location: https://github.com/CWorthy-ocean/cstar_blueprint_roms_marbl_example.git
    branch: main
    filter:
      directory: wio-toy-domain/compile_time_code
      files:
        - bgc.opt
        - bulk_frc.opt
        - cdr_frc.opt
        - cppdefs.opt
        - diagnostics.opt
  

In [1]:
!cstar blueprint check wio_toy_blueprint.yaml

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.
The blueprint is valid


In [3]:
!rm -rf output

In [7]:
!cstar blueprint run wio_toy_blueprint.yaml

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.
Executing blueprint in a worker service
2026-02-06 17:21:09,958 [DEBUG] - worker.py:459 - Job config: JobConfig(account_id='None', walltime='0:10:00', job_name='None', priority='regular')
2026-02-06 17:21:09,958 [DEBUG] - worker.py:460 - Simulation runner service config: as_service=True loop_delay=5.0 health_check_frequency=None log_level=10 health_check_log_threshold=10 name='SimulationRunner' max_health_check_latency=1.0
2026-02-06 17:21:09,958 [DEBUG] - worker.py:461 - Simulation request: BlueprintRequest(blueprint_uri='wio_toy_blueprint.yaml', stages=[<SimulationStages.SETUP: 'setup'>, <SimulationStages.BUILD: 'build'>, <SimulationStages.PRE_RUN: 'pre_run'>, <SimulationStages.RUN: 'run'>, <SimulationStages.POST_RUN: 'post_run'>])
2026-02-06 17:21:09,958 [DEBUG] - worker.py:462 - os.environ: environ({'JPY_SESSION_NAME': '/Users/sam.maticka/workdir/cstar_blueprint_roms_marbl_example/wio-t